In [1]:
import os
os.chdir('../..')

In [193]:
import pandas as pd
import numpy as np

from datetime import datetime, date
import matplotlib.pyplot as plt

import json

data = 'data/'

In [4]:
atms = pd.read_csv(data + 'atms.small.csv')
clients = pd.read_csv(data + 'clients.small.csv')
companies = pd.read_csv(data + 'companies.small.csv')
transactions = pd.read_csv(data + 'transactions.small.csv')

In [281]:
months = {}
days = {}
counts = 0
dates = []
for i in transactions.index:
    date = transactions.loc[i]['date']
    month = int(date.split('/')[1])
    day = int(date.split('/')[0])
    
    if month not in months.keys():
        months[month] = 0
    
    months[month] += 1
    if day not in days.keys():
        days[day] = 0
    
    days[day] += 1
    
    if day > 12 and month > 12:
        counts += 1

In [282]:
counts/len(transactions)

0.11905019472090005

In [54]:
months

{1: 618,
 2: 622,
 3: 663,
 4: 527,
 5: 518,
 6: 661,
 7: 564,
 8: 570,
 9: 627,
 10: 450,
 11: 505,
 12: 507,
 13: 759,
 14: 552,
 15: 575,
 16: 612,
 17: 843,
 18: 675,
 19: 579,
 20: 922,
 21: 612,
 22: 709,
 23: 752,
 24: 556,
 25: 503,
 26: 474,
 27: 646,
 28: 515,
 29: 558,
 30: 579,
 31: 235}

In [57]:
days

{1: 1246,
 2: 1301,
 3: 1188,
 4: 1265,
 5: 1283,
 6: 1368,
 7: 1241,
 8: 1317,
 9: 1307,
 10: 1270,
 11: 1284,
 12: 1542,
 13: 49,
 14: 52,
 15: 43,
 16: 49,
 17: 46,
 18: 40,
 19: 45,
 20: 45,
 21: 37,
 22: 44,
 23: 41,
 24: 233,
 25: 524,
 26: 573,
 27: 1055}

## Extract time and flow patterns

In [117]:
threshold_minutes = 15
threshold_amount = 0.1

In [277]:
groups = {'source': {'time': [], 'flow': []},
          'target': {'time': [], 'flow': []}}

ids = {'source': transactions['source'].unique(), 'target': transactions['target'].unique()}

for key in ids.keys():
    print('Check for {}'.format(key))
    for id_ in ids[key]:
        df = transactions[transactions[key] == id_]
        df.index = range(len(df))

        times = np.zeros((len(df), len(df)), dtype=np.bool)
        amounts = np.zeros((len(df), len(df)), dtype=np.bool)


        for j in range(len(df)):
            for k in range(j, len(df)):
                # Time diff
                tj = datetime.strptime(df.loc[j]['time'], '%H:%M:%S').time()
                tk = datetime.strptime(df.loc[k]['time'], '%H:%M:%S').time()

                diff = np.abs(((datetime.combine(date.today(), tj) - datetime.combine(date.today(), tk)).total_seconds()))

                if diff < threshold_minutes*60:
                    times[j][k] = True
                    times[k][j] = True
                else:
                    times[j][k] = False
                    times[k][j] = False

                # Amount diff

                amj = df.loc[j]['amount']
                amk = df.loc[k]['amount']

                diff = np.abs(amj-amk)
                avg = np.mean([amj, amk])

                if diff < threshold_amount*avg:
                    amounts[j][k] = True
                    amounts[k][j] = True
                else:
                    amounts[j][k] = False
                    amounts[k][j] = False

        mat = amounts*times
        
        j=0
        while j <= len(df)-1:
            arr = [i for i, x in enumerate(mat[j, :]) if x]
            j = arr[-1]+1
            if len(arr) > 2:
                linked = [df.loc[k]['id'] for k in arr]
                if len(df[df['id'].isin(linked)]['date'].unique()) == 1:
                    groups[key]['flow'].append(linked)
                else:
                    groups[key]['time'].append(linked) 

Check for source
Check for target


In [398]:
for key in groups.keys():
    print('{}:'.format(key))
    for type_ in groups[key].keys():
        print('  {}: {}'.format(type_, len(groups[key][type_])))

source:
  time: 84
  flow: 258
target:
  time: 89
  flow: 241


In [ ]:
with open(data + 'groups.json', 'w') as outfile:
    json.dump(groups, outfile)

# Check for similar transactions

In [403]:
with open(data + 'groups.json', 'r') as outfile:
    groups = json.load(outfile)

In [404]:
for key in groups.keys():
    for type_ in groups[key].keys():
        
        ids = []
        dup = []
        for i, g in enumerate(groups[key][type_]):
            df = transactions[transactions['id'].isin(g)]
            id_ = df[key].iloc[0]

            if id_ in ids:
                dup.append([ids.index(id_), i])
            ids.append(id_)  
            
        merge = []
        for pair in dup:
            amount1 = np.mean(transactions[transactions['id'].isin(groups[key][type_][pair[0]])]['amount'])
            amount2 = np.mean(transactions[transactions['id'].isin(groups[key][type_][pair[1]])]['amount'])

            delta = np.abs(amount1-amount2)

            if delta < threshold_amount*np.mean([amount1, amount2]):
                merge.append(pair)
                
        to_del = []
        for pair in merge:
            groups[key][type_][pair[0]] = list(np.append(groups[key][type_][pair[0]], groups[key][type_][pair[1]]))

            to_del.append(pair[1])
            
        for index in sorted(to_del, reverse=True):
            del groups[key][type_][index]

In [405]:
for key in groups.keys():
    print('{}:'.format(key))
    for type_ in groups[key].keys():
        print('  {}: {}'.format(type_, len(groups[key][type_])))

source:
  time: 83
  flow: 256
target:
  time: 89
  flow: 241


In [406]:
with open(data + 'groups.json', 'w') as outfile:
    json.dump(groups, outfile)

# Extract time patterns now

In [407]:
groups['source']['time']

[['572c7694-ff53-47ca-b9b4-3de34224c36d',
  '4b0a1020-dfb9-430d-9abc-f629d0fd9271',
  '457cd282-cc6c-4b7d-b24a-5786af060fd6',
  'acfb980c-5508-4c76-ad5f-edcfc6c31788',
  '58453044-4a20-4c0e-bb33-61001eb6dd4f',
  'dec04d74-5b39-4360-9348-d442a2dc90de',
  'dcac5c72-b6e4-494f-872c-5e9f12256861',
  '7b077fc3-5f55-4c40-9295-5686f38ac0ee',
  '7db0ca56-6de7-49d6-a8e7-dc17fab327fd',
  'a966c280-8965-4261-9e10-5c9aeecb7224',
  'd6559bbe-b7d5-4cf6-ac10-dcc29375438c',
  '630fda63-8a79-455c-beeb-1dd5000f273a',
  '1c1ff189-680d-48f6-a29f-3fabc6ad0976',
  '7cf7b8fd-010c-4683-9129-08ea42fdf65e',
  '58d8aefd-c0ac-4f83-9470-f3473a6d82ce',
  'c90541bf-e9ab-48a8-972d-e7a1129bed3b',
  'e371d4b7-f60c-4fc9-af63-db90305053e6',
  '587d3b30-2fa8-4a4e-959c-1faa81195146',
  '9c9ebede-d486-4a9f-a10a-a442139e0884',
  'c8408e37-6f65-45be-8fd6-af031c4b29b1',
  '1dce1737-6f4e-43e9-aec9-b88a3fe3ce9f',
  'fd505a61-b979-4157-88e0-6c8017a9067c',
  'b5a6de05-eb04-4dfe-a9c0-01f173c56ab1',
  'c17cf7fc-30cb-4bbd-8af8-1bc97ee